In [1]:
import numpy as np
import dlib
from imutils import face_utils
import os
import cv2

In [3]:
from keras.models import load_model

model = load_model("covid_dlib.h5", compile= False)

In [6]:
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_81_face_landmarks.dat")
cap = cv2.VideoCapture(0)

while (cap.isOpened()):
    
    ret, frame=cap.read()
    if not ret:
        break  
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    subjects= detect(gray, 1)
    if len(subjects) !=0:
        for subject in subjects: 
            shape = predict(gray, subject)
            shape = face_utils.shape_to_np(shape)
            (x, y, w, h) = face_utils.rect_to_bb(subject)

            roi_gray=gray[y-25:y+h+10,x-25:x+w+10]

            if len(roi_gray)==0:
                continue

            result=np.argmax(model.predict(cv2.resize(roi_gray,(128,128)).reshape(-1,128,128,1)), axis=1)

            color = (0,255,0) if result==1 else (0,0,255)
            
            cv2.rectangle(frame,(x,y),(x+w,y+h),color ,4) 


            
 
            cv2.putText(frame,"Mask" if result==1 else "No Mask", (x,y-20), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    
    cv2.imshow('Mask', frame)
        
    if cv2.waitKey(1)==13:
        break
    
cap.release()                    
cv2.destroyAllWindows()       